## Load Data

In [1]:
function parseData(filename)
    rawData = readlines(filename)
    splitRaw=findfirst(x->occursin("fold",x),rawData)
    
    points = parse.(Int,hcat(split.(rawData[1:splitRaw-2],",")...))
    
    nfolds = length(rawData)-splitRaw+1
    folds = Array{Int}(undef,nfolds,2);
    for i in 1:nfolds
        dummyString = split(rawData[splitRaw+i-1])[end]
        foldInstruction = split(dummyString,"=")
        if foldInstruction[1] == "x"
            folds[i,:] = [0,parse(Int,foldInstruction[2])]
        else
            folds[i,:] = [1,parse(Int,foldInstruction[2])]
        end
    end
    return(points,folds)
end

parseData (generic function with 1 method)

In [2]:
points,folds = parseData("dottedPaper.txt")

([309 32 … 954 960; 320 761 … 841 278], [0 655; 1 447; … ; 1 13; 1 6])

In [3]:
pointsTest,foldsTest = parseData("dottedPaperTest.txt")

([6 0 … 8 9; 10 14 … 10 0], [1 7; 0 5])

## Part 1

In [4]:
maximum(pointsTest[1,:]),maximum(pointsTest[2,:])

(10, 14)

In [5]:
foldsTest

2×2 Array{Int64,2}:
 1  7
 0  5

In [38]:
function createSheet(points,folds)
    nx = folds[findfirst(x->x==0,folds[:,1]),2]*2+1
    ny = folds[findfirst(x->x==1,folds[:,1]),2]*2+1
    
    sheet = zeros(Int,ny,nx)
    npoints = size(points)[2]
    for i in 1:npoints
        #assign points as ones, taking shift from 0 to 1 based indexing into account
        sheet[points[2,i]+1,points[1,i]+1] = 1
    end
    return sheet
end

function foldOrigamiOnce(sheet_init,fold)
    #fold left
    sheet = copy(sheet_init)
    ny,nx = size(sheet)
    if fold[1] == 0
        for i in 1:fold[2],j in 1:ny
            if sheet[j,nx-(i-1)] == 1
                sheet[j,i] = 1
            end
        end
        return sheet[:,1:fold[2]]
    #fold up    
    else
       for i in 1:fold[2],j in 1:nx
            if sheet[ny-(i-1),j] == 1
                sheet[i,j] = 1
            end
        end
        return sheet[1:fold[2],:] 
    end
end

foldOrigamiOnce (generic function with 1 method)

In [40]:
sheetTest = createSheet(pointsTest,foldsTest);

In [41]:
foldOnetest = foldOrigamiOnce(sheetTest,foldsTest[1,:])

7×11 Array{Int64,2}:
 1  0  1  1  0  0  1  0  0  1  0
 1  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  1
 1  0  0  0  1  0  0  0  0  0  0
 0  1  0  1  0  0  1  0  1  1  1
 0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0

In [42]:
sum(sheetTest)

18

In [43]:
sum(foldOnetest)

17

In [11]:
foldOnetest = foldOrigamiOnce(foldOnetest,foldsTest[2,:])

7×5 Array{Int64,2}:
 1  1  1  1  1
 1  0  0  0  1
 1  0  0  0  1
 1  0  0  0  1
 1  1  1  1  1
 0  0  0  0  0
 0  0  0  0  0

In [44]:
sheet = createSheet(points,folds)
foldOne = foldOrigamiOnce(sheet,folds[1,:]);

In [45]:
size(sheet)

(895, 1311)

In [46]:
sum(sheet)

802

In [47]:
sum(foldOne)

669

## Part 2

In [70]:
function foldOrigami(sheet,folds)
    nfolds = size(folds)[1]
    newSheet = copy(sheet)
    for i in 1:nfolds
        newSheet = foldOrigamiOnce(newSheet,folds[i,:])
    end
    return newSheet
end

function prettyPrintOrigami(sheet)
    ni,nj = size(sheet)
    for i in 1:ni
        for j in 1:nj
            if sheet[i,j] == 1
                print("x")
            else
                print(" ")
            end
        end
        print("\n")
    end
end

prettyPrintOrigami (generic function with 1 method)

In [71]:
origamiTest = foldOrigami(sheetTest,foldsTest)

7×5 Array{Int64,2}:
 1  1  1  1  1
 1  0  0  0  1
 1  0  0  0  1
 1  0  0  0  1
 1  1  1  1  1
 0  0  0  0  0
 0  0  0  0  0

In [72]:
prettyPrintOrigami(origamiTest)

xxxxx
x   x
x   x
x   x
xxxxx
     
     


In [73]:
origami = foldOrigami(sheet,folds);

In [74]:
prettyPrintOrigami(origami)

x  x xxxx xxxx xxxx  xx  x  x  xx    xx 
x  x x    x       x x  x x  x x  x    x 
x  x xxx  xxx    x  x    x  x x       x 
x  x x    x     x   x    x  x x       x 
x  x x    x    x    x  x x  x x  x x  x 
 xx  xxxx x    xxxx  xx   xx   xx   xx  
